In [1]:
import torch
import gymnasium as gym

class DQN(torch.nn.Module):
    """Neural network that approximates Q-values for reinforcement learning."""
    
    def __init__(self, state_size=8, action_size=4, hidden_size=64):
        """Initialize three-layer network: input -> hidden -> output."""
        super(DQN, self).__init__()
        self.layer1 = torch.nn.Linear(state_size, hidden_size)
        self.layer2 = torch.nn.Linear(hidden_size, hidden_size)
        self.layer3 = torch.nn.Linear(hidden_size, action_size)

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        """Forward pass: state -> Q-values for each action."""
        x = torch.relu(self.layer1(state))
        x = torch.relu(self.layer2(x))
        return self.layer3(x)

In [2]:
from collections import deque
import numpy as np
import random


class ReplayBuffer:
    def __init__(self, max_size: int = 10_000):
        """Initialize replay memory buffer.
        
        Args:
            max_size: Maximum buffer capacity (default 10000)
        """
        self.memory = deque(maxlen=max_size)
        self.max_size = max_size

    def store_transition(self, state, action, reward, next_state, done):
        """Store transition in memory.
        
        Args:
            state: Current state
            action: Action taken
            reward: Reward received
            next_state: Next state
            done: Episode termination flag
        """
        transition = (state, action, reward, next_state, done)
        self.memory.append(transition)

    def get_batch(self, batch_size: int):
        """Get random batch of transitions from memory.
        
        Args:
            batch_size: Size of batch to retrieve
            
        Returns:
            Tuple of (states, actions, rewards, next_states, dones)
        """
        if batch_size > len(self.memory):
            raise ValueError(f"Not enough samples in memory: {len(self.memory)}")
            
        # Get random sample
        batch = random.sample(self.memory, batch_size)
        
        # Unpack into separate lists
        states, actions, rewards, next_states, dones = zip(*batch)
        
        # Convert to numpy arrays
        return (
            np.array(states),
            np.array(actions),
            np.array(rewards),
            np.array(next_states),
            np.array(dones)
        )

    def __len__(self):
        """Return current memory size."""
        return len(self.memory)

    @property
    def is_ready(self):
        """Check if memory is ready for sampling."""
        return len(self.memory) > 0

In [3]:
import torch
import numpy as np
import random
from typing import Tuple

class DQNAgent:
    def __init__(self, state_size=8, action_size=4, hidden_size=64, 
                 learning_rate=1e-3, gamma=0.99, buffer_size=10000, batch_size=64):
        """Initialize DQN Agent."""
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.action_size = action_size
        self.gamma = gamma
        self.batch_size = batch_size
        
        # Initialize networks
        self.q_network = self._init_network(state_size, action_size, hidden_size)
        self.target_network = self._init_network(state_size, action_size, hidden_size)
        self._sync_target_network()
        
        self.optimizer = torch.optim.Adam(self.q_network.parameters(), lr=learning_rate)
        self.memory = ReplayBuffer(buffer_size)

    def _init_network(self, state_size: int, action_size: int, hidden_size: int) -> torch.nn.Module:
        """Initialize and return network on correct device."""
        return DQN(state_size, action_size, hidden_size).to(self.device)

    def _sync_target_network(self):
        """Copy Q-network weights to target network."""
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()

    def act(self, state: np.ndarray, epsilon: float = 0.0) -> int:
        """Choose action using epsilon-greedy policy."""
        if random.random() > epsilon:
            return self._get_best_action(state)
        return random.randrange(self.action_size)

    def _get_best_action(self, state: np.ndarray) -> int:
        """Get action with highest Q-value."""
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        
        self.q_network.eval()
        with torch.no_grad():
            q_values = self.q_network(state)
        self.q_network.train()
        
        return np.argmax(q_values.cpu().numpy())

    def step(self, state: np.ndarray, action: int, reward: float, next_state: np.ndarray, done: bool):
        """Store experience and potentially update model."""
        self.memory.store_transition(state, action, reward, next_state, done)
        
        if len(self.memory) > self.batch_size:
            self._learn()

    def _learn(self):
        """Update Q-network using experience replay."""
        # Get batch
        batch = self._get_batch()
        states, actions, rewards, next_states, dones = batch
        
        # Get current Q values
        current_q = self.q_network(states).gather(1, actions.unsqueeze(-1)).squeeze(-1)
        
        # Get target Q values
        with torch.no_grad():
            next_q = self.target_network(next_states).max(1)[0]
            target_q = rewards + self.gamma * next_q * (1 - dones)
        
        # Update network
        loss = torch.nn.functional.mse_loss(current_q, target_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def _get_batch(self) -> Tuple:
        """Get and process batch from replay buffer."""
        states, actions, rewards, next_states, dones = self.memory.get_batch(self.batch_size)
        
        return (
            torch.from_numpy(states).float().to(self.device),
            torch.tensor(actions).long().to(self.device),
            torch.tensor(rewards).float().to(self.device),
            torch.from_numpy(next_states).float().to(self.device),
            torch.tensor(dones).float().to(self.device)
        )

    def update_target_network(self):
        """Update target network weights."""
        self._sync_target_network()

In [4]:
import gymnasium as gym
import numpy as np
from collections import deque

class DQNTrainer:
    def __init__(self, env, agent, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
        self.env = env
        self.agent = agent
        self.eps = eps_start
        self.eps_end = eps_end
        self.eps_decay = eps_decay
        self.best_score = float('-inf')
        
    def train(self, n_episodes=3000, target_update=10, solved_threshold=190 ):
        scores = []
        window_size = 100
        min_stable_episodes = 300
        
        for episode in range(1, n_episodes + 1):
            score = self._run_episode()
            scores.append(score)
            
            # Update epsilon and target network
            self.eps = max(self.eps_end, self.eps_decay * self.eps)
            if episode % target_update == 0:
                self.agent.update_target_network()
            
            # Check stability after collecting enough episodes
            if episode >= min_stable_episodes:
                # Calculate averages for last 300 episodes in windows of 100
                recent_means = []
                for i in range(episode-min_stable_episodes, episode-window_size+1, window_size):
                    window_mean = np.mean(scores[i:i+window_size])
                    recent_means.append(window_mean)
                
                # Check if minimum average is above threshold
                if min(recent_means) >= solved_threshold:
                    print(f"\nEnvironment solved in {episode} episodes!")
                    print(f"Last three averages: {[f'{x:.2f}' for x in recent_means]}")
                    break
            
            # Update tracking and print progress
            if len(scores) >= window_size:
                current_avg = np.mean(scores[-window_size:])
                self.best_score = max(self.best_score, current_avg)
            else:
                current_avg = np.mean(scores)
            
            self._print_progress(episode, current_avg)
            
        return scores, self.best_score
    
    def _run_episode(self):
        state, _ = self.env.reset()
        total_reward = 0
        
        while True:
            action = self.agent.act(state, self.eps)
            next_state, reward, terminated, truncated, _ = self.env.step(action)
            done = terminated or truncated
            
            self.agent.step(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
            
            if done:
                break
                
        return total_reward
        
    def _print_progress(self, episode, avg_score):
        print(f"\rEpisode {episode} | Avg Score: {avg_score:.2f} | Best: {self.best_score:.2f} | Epsilon: {self.eps:.3f}", end="")
        if episode % 100 == 0:
            print()

def train_lunar_lander():
    env = gym.make('LunarLander-v3')
    agent = DQNAgent(
        state_size=env.observation_space.shape[0],
        action_size=env.action_space.n,
        hidden_size=64,
        learning_rate=1e-3
    )
    
    trainer = DQNTrainer(
        env=env,
        agent=agent,
        eps_start=1.0,
        eps_end=0.01,
        eps_decay=0.998
    )
    
    scores, best_score = trainer.train(
        n_episodes=3000,
        target_update=10,
        solved_threshold=200
    )
    
    return agent, scores, best_score

if __name__ == "__main__":
    agent, scores, best_score = train_lunar_lander()

Episode 100 | Avg Score: -138.48 | Best: -138.48 | Epsilon: 0.819
Episode 200 | Avg Score: -97.41 | Best: -97.19 | Epsilon: 0.67088
Episode 300 | Avg Score: -60.67 | Best: -60.67 | Epsilon: 0.548
Episode 400 | Avg Score: -54.57 | Best: -48.19 | Epsilon: 0.449
Episode 500 | Avg Score: -1.84 | Best: -0.69 | Epsilon: 0.36879
Episode 600 | Avg Score: 10.92 | Best: 20.99 | Epsilon: 0.301
Episode 700 | Avg Score: 39.29 | Best: 39.29 | Epsilon: 0.246
Episode 800 | Avg Score: 75.12 | Best: 87.40 | Epsilon: 0.202
Episode 900 | Avg Score: 57.87 | Best: 87.40 | Epsilon: 0.165
Episode 1000 | Avg Score: 76.96 | Best: 87.40 | Epsilon: 0.135
Episode 1100 | Avg Score: -34.78 | Best: 87.40 | Epsilon: 0.111
Episode 1200 | Avg Score: 122.53 | Best: 122.53 | Epsilon: 0.091
Episode 1300 | Avg Score: 184.43 | Best: 186.43 | Epsilon: 0.074
Episode 1400 | Avg Score: 179.88 | Best: 195.83 | Epsilon: 0.061
Episode 1500 | Avg Score: 96.77 | Best: 195.83 | Epsilon: 0.0500
Episode 1600 | Avg Score: 127.11 | Best: 